In [2]:
from src.classes.network import Network
from src.classes.node import Node
from multiprocessing import Pool
import os


In [16]:
# voorbeeld

num_nodes = 100
correlation = -2
update_fraction = 0.1
starting_distribution = 0.5     # L / R ratio (niet per se nodig maar kan misschien leuk zijn om te varieern)
p = 0.2
k = 2

network = Network(num_nodes, correlation, update_fraction, starting_distribution, seed=33, p=p)
number_of_alterations = 0
for round in range(100):
    network.update_round()
    number_of_alterations += network.alterations
print(number_of_alterations)

A random network is initialized with p: 0.2 and 100 nodes
42


In [ ]:
from src.classes.network import Network
from src.viusalization import plot_network

def create_network():
    netwerkje = Network(1000, -1, p=0.02)
    animation = plot_network(netwerkje)
create_network()


A random network is initialized with p: 0.02 and 1000 nodes
between frames 0, 0 there were 21 alterations made (208 network updates made)
between frames 0, 0 there were 21 alterations made (321 network updates made)


KeyboardInterrupt: 

In [1]:
from src.experimentation import experiment_a

pars = []
seed = 33
for i in range(10):
    seed += i*10
    netwerkje = Network(100, -1, p=0.02, seed=seed)
    pars.append(netwerkje)
    
PROCESSES = 10
assert PROCESSES < os.cpu_count(), "Lower the number of processes (PROCESSES)"
with Pool(PROCESSES) as pool:
        assert PROCESSES < os.cpu_count(), "Lower the number of processes (PROCESSES)"
        print(f"Starting parallel execution for {experiment_a} schedule")
        results = pool.map(experiment_a, pars)

NameError: name 'Network' is not defined